In [1]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import seaborn as sns
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.callbacks import ReduceLROnPlateau

In [2]:

model = tf.keras.models.Sequential([
                                    # The first convolution
                                    # Input image has 3 bytes color
                                    tf.keras.layers.Conv2D(16, (3, 3), activation="relu", input_shape=(300, 300, 3)),
                                    tf.keras.layers.MaxPooling2D(2, 2),
                                    # The second convolution
                                    tf.keras.layers.Conv2D(32, (3, 3), activation="relu"),
                                    tf.keras.layers.MaxPooling2D(2, 2),
                                    # The third convolution
                                    tf.keras.layers.Conv2D(64, (3, 3), activation="relu"),
                                    tf.keras.layers.MaxPooling2D(2, 2),
                                    # The fourth convolution
                                    tf.keras.layers.Conv2D(64, (3, 3), activation="relu"),
                                    tf.keras.layers.MaxPooling2D(2, 2),
                                    # The fifth convolution
                                    tf.keras.layers.Conv2D(128, (3, 3), activation="relu"),
                                    tf.keras.layers.MaxPooling2D(2, 2),
                                    # The sixth convolution
                                    tf.keras.layers.Conv2D(128, (3, 3), activation="relu"),
                                    tf.keras.layers.MaxPooling2D(2, 2),
                                    # Flatten the results to feed in Deep Neural Network
                                    tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(512, activation="relu"),
                                    tf.keras.layers.Dense(1, activation="sigmoid")                                   
])

2022-11-11 07:27:35.492202: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [3]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 298, 298, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 149, 149, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 147, 147, 32)      4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 73, 73, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 71, 71, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 35, 35, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 33, 33, 64)        3

In [4]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [5]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode="nearest")

validation_datagen = ImageDataGenerator(rescale=1./255)

In [6]:
train_generator = train_datagen.flow_from_directory(
    '../input/good-guysbad-guys-image-data-set/train',
    target_size=(300, 300),
    batch_size=64,
    class_mode="binary"
)

# Flow validation images in batches of 32 using validation_datagen generator
validation_generator = validation_datagen.flow_from_directory(
    '../input/good-guysbad-guys-image-data-set/test',
    target_size=(300, 300),
    batch_size=32, 
    class_mode="binary"
)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)

# model.fit(x = train_generator,validation_data = validation_generator, callbacks= reduce_lr)

history = model.fit(train_generator,
                         steps_per_epoch = 50,
                         epochs = 10,
                         validation_data = validation_generator,
                         validation_steps = 8)

Found 11220 images belonging to 2 classes.
Found 600 images belonging to 2 classes.


2022-11-11 07:27:43.400118: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/10
50/50 [==============================] - 154s 3s/step - loss: 0.6890 - accuracy: 0.5269 - val_loss: 0.6543 - val_accuracy: 0.6094
Epoch 2/10
50/50 [==============================] - 150s 3s/step - loss: 0.6651 - accuracy: 0.5844 - val_loss: 0.5794 - val_accuracy: 0.6094
Epoch 3/10
50/50 [==============================] - 147s 3s/step - loss: 0.6574 - accuracy: 0.5745 - val_loss: 0.5631 - val_accuracy: 0.6680
Epoch 4/10
50/50 [==============================] - 147s 3s/step - loss: 0.6325 - accuracy: 0.6303 - val_loss: 0.5152 - val_accuracy: 0.7500
Epoch 5/10
50/50 [==============================] - 147s 3s/step - loss: 0.6165 - accuracy: 0.6575 - val_loss: 0.5232 - val_accuracy: 0.7617
Epoch 6/10
50/50 [==============================] - 143s 3s/step - loss: 0.5859 - accuracy: 0.6744 - val_loss: 0.4591 - val_accuracy: 0.7852
Epoch 7/10
50/50 [==============================] - 145s 3s/step - loss: 0.5677 - accuracy: 0.6978 - val_loss: 0.4478 - val_accuracy: 0.7852
Epoch 8/10
50

In [8]:
from keras.preprocessing import image
import numpy as np
# Predicting Images
path = input()
img = image.load_img(path, target_size=(300,300))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)

images = np.vstack([x])
classes = model.predict(images, batch_size=10)
print(classes[0]) 
if classes[0] > 0.5:
    print(img, "is  bad guy")
else:
    print(img, "is  good guy")

 ../input/dfghjk/Screenshot 2022-04-18 at 1.20.44 PM.png


[1.]
<PIL.Image.Image image mode=RGB size=300x300 at 0x7F2F6C976950> is  bad guy
